# Overview

This notebook contains my solutions for **<a href="https://adventofcode.com/2024" target="_blank">Advent of Code 2024</a>**.

A few notes...
- The source for this notebook source lives in my GitHub repo, <a href="https://github.com/derailed-dash/Advent-of-Code/blob/master/src/AoC_2024/Dazbo's_Advent_of_Code_2024.ipynb" target="_blank">here</a>.
- You can run this Notebook wherever you like. For example, you could...
  - Run it locally, in your own Jupyter environment.
  - Run it in a cloud-based Jupyter environment, with no setup required on your part!  For example, with **Google Colab**: <br><br><a href="https://colab.research.google.com/github/derailed-dash/Advent-of-Code/blob/master/src/AoC_2024/Dazbo's_Advent_of_Code_2024.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Google Colab"/></a><br>
  - For more ways to run Jupyter Notebooks, check out [my guide](https://medium.com/python-in-plain-english/five-ways-to-run-jupyter-labs-and-notebooks-23209f71e5c0).
- **To run the notebook, execute the cells in the [Setup](#Setup) section, as described below. Then you can run the code for any given day.**
- Be mindful that the first time you run this notebook, you will need to **obtain your AoC session key** and store it, if you have not done so already. This allows the notebook to automatically retrieve your input data. (See the guidance in the **[Get Access to Your AoC Data](#Get-Access-to-Your-AoC-Data)** section for details.)
- Use the navigation menu on the left to jump to any particular day.
- All of my AoC solutions are documented in my <a href="https://aoc.just2good.co.uk/" target="_blank">AoC Python Walkthrough site</a>.

# Setup

You need to run all cells in this section, before running any particular day solution.

## Packages and Imports

Here we use `pip` to install the packages used by my solutions in this event. After installing the packages, you may need to restart your Jupyter kernel in order for the packages to be detected and remove any linting errors.

In [1]:
%pip install --upgrade --no-cache-dir \
    jupyterlab-lsp ipykernel \
    matplotlib pandas networkx  sympy \
    dazbo-commons \
    python-dotenv tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from __future__ import annotations
import ast
import copy
import heapq
import logging
import math
import operator
import os
import platform
import re
from collections import Counter, deque, defaultdict
from dataclasses import asdict, dataclass, field
from enum import Enum, auto
from functools import cache, reduce
from itertools import combinations, count, cycle, permutations
from getpass import getpass
from pathlib import Path

# Third-party imports
import dazbo_commons as dc  # my own utility library, which includes things like coloured logging
import requests
import sympy
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from IPython.core.display import Markdown
from IPython.display import display
from tqdm.notebook import tqdm

## Logging and Output

Set up a new logger that uses `ColouredFormatter`, such that we have coloured logging.  The log colour depends on the logging level.

In [3]:
# Setup logger
YEAR = 2024
APP_NAME = "aoc" + str(YEAR)
logger = dc.retrieve_console_logger(APP_NAME)
logger.setLevel(logging.DEBUG)
logger.info("Logger initialised.")
logger.debug("Debugging enabled")

17:14:12.476:aoc2024 - INF: Logger initialised.
17:14:12.477:aoc2024 - DBG: Debugging enabled


## Install Packages

- [ffmpeg](https://ffmpeg.org/): in order to render video output, i.e. for visualisations.
- graphviz: for visualising graphs

In [4]:
import subprocess

def run_command(command):
    """Run a shell command and print its output in real-time."""
    process = subprocess.Popen(
        command, 
        shell=True, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.PIPE
    )
    
    # Read and print the output line by line
    if process.stdout is not None:
        for line in iter(process.stdout.readline, b''):
            logger.info(line.decode().strip())
        process.stdout.close()
        
    process.wait()
    
def install_software(appname: str):
    os_name = platform.system()
    logger.info(f"Installing {appname} on {os_name}...")
    
    # Mapping operating systems to their respective installation commands
    command_map = {
        "Windows": f"winget install {appname} --silent --no-upgrade",
        "Linux": f"apt -qq -y install {appname}",
        "Darwin": f"brew install {appname}"
    }
    command = command_map.get(os_name)
    if command:
        run_command(command)
    else:
        logger.error(f"Unsupported operating system: {os_name}")

def check_installed(app_exec: str) -> bool:    
    appname, *arg = app_exec.split()
    arg = " ".join(arg)
    logger.debug(f"Checking if {appname} is installed")
    
    try:
        output = subprocess.check_output([appname, arg], stderr=subprocess.STDOUT)
        logger.debug(f"{appname} version: {output.decode().strip()}")
        logger.debug(f"{appname} is already installed.")
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        logger.debug(f"{appname} is not installed or absent from path.")
        
    return False

apps = [ ("ffmpeg", "ffmpeg -version"),
         ("graphviz", "dot --version") ]
          
for app_install, app_exec in apps:
    if not check_installed(app_exec):
        install_software(app_install)


17:14:12.498:aoc2024 - DBG: Checking if ffmpeg is installed
17:14:12.592:aoc2024 - DBG: ffmpeg version: ffmpeg version 6.1-full_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
built with gcc 12.2.0 (Rev10, Built by MSYS2 project)
configuration: --enable-gpl --enable-version3 --enable-static --pkg-config=pkgconf --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --e

## Load Helpers and Useful Classes

Now we load a bunch of helper functions and classes.

### Locations

Where any input and output files get stored.

<img src="https://aoc.just2good.co.uk/assets/images/notebook-content-screenshot.png" width="320" />


### Retrieve the Input Data

This works by using your unique session cookie to retrieve your input data. E.g. from a URL like:

`https://adventofcode.com/2024/day/1/input`

In [5]:
##################################################################
# Retrieving input data
##################################################################

def write_puzzle_input_file(year: int, day, locations: dc.Locations):
    """ Use session key to obtain user's unique data for this year and day.
    Only retrieve if the input file does not already exist.
    Return True if successful.
    Requires env: AOC_SESSION_COOKIE, which can be set from the .env.
    """
    if os.path.exists(locations.input_file):
        logger.debug("%s already exists", os.path.basename(locations.input_file))
        return os.path.basename(locations.input_file)

    session_cookie = os.getenv('AOC_SESSION_COOKIE')
    if not session_cookie:
        raise ValueError("Could not retrieve session cookie.")

    logger.info('Session cookie retrieved: %s...%s', session_cookie[0:6], session_cookie[-6:])

    # Create input folder, if it doesn't exist
    if not locations.input_dir.exists():
        locations.input_dir.mkdir(parents=True, exist_ok=True)

    url = f"https://adventofcode.com/{year}/day/{day}/input"
    
    # Don't think we need to set a user-agent
    # headers = {
    #     "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
    # }
    cookies = { 
        "session": session_cookie
    }
    response = requests.get(url, cookies=cookies, timeout=5)

    data = ""
    if response.status_code == 200:
        data = response.text

        with open(locations.input_file, 'w') as file:
            logger.debug("Writing input file %s", os.path.basename(locations.input_file))
            file.write(data)
            return data
    else:
        raise ValueError(f"Unable to retrieve input data.\n" +
                         f"HTTP response: {response.status_code}\n" +
                         f"{response.reason}: {response.content.decode('utf-8').strip()}")


### Testing

A really simple function for testing that our solution produces the expected test output. If the `validate()` call fails, then execution will stop.

In [6]:
def validate(test, answer):
    """
    Args:
        test: the answer given by our solution
        answer: the expected answer, e.g. from instructions
    """
    if test != answer:
        raise AssertionError(f"{test} != {answer}")

### Useful Helper Classes

In [7]:
#################################################################
# POINTS, VECTORS AND GRIDS
#################################################################

@dataclass(frozen=True)
class Point:
    """ Class for storing a point x,y coordinate """
    x: int
    y: int

    def __add__(self, other: Point):
        return Point(self.x + other.x, self.y + other.y)

    def __mul__(self, other: Point):
        """ (x, y) * (a, b) = (xa, yb) """
        return Point(self.x * other.x, self.y * other.y)

    def __sub__(self, other: Point):
        return self + Point(-other.x, -other.y)

    def __lt__(self, other):
        # Arbitrary comparison logic
        return (self.x, self.y) < (other.x, other.y)
    
    def yield_neighbours(self, include_diagonals=True, include_self=False):
        """ Generator to yield neighbouring Points """

        deltas: list
        if not include_diagonals:
            deltas = [vector.value for vector in Vectors if abs(vector.value[0]) != abs(vector.value[1])]
        else:
            deltas = [vector.value for vector in Vectors]

        if include_self:
            deltas.append((0, 0))

        for delta in deltas:
            yield Point(self.x + delta[0], self.y + delta[1])

    def neighbours(self, include_diagonals=True, include_self=False) -> list[Point]:
        """ Return all the neighbours, with specified constraints.
        It wraps the generator with a list. """
        return list(self.yield_neighbours(include_diagonals, include_self))

    def get_specific_neighbours(self, directions: list[Vectors]) -> list[Point]:
        """ Get neighbours, given a specific list of allowed locations """
        return [(self + Point(*vector.value)) for vector in list(directions)]

    @staticmethod
    def manhattan_distance(a_point: Point) -> int:
        """ Return the Manhattan distance value of this vector """
        return sum(abs(coord) for coord in asdict(a_point).values())

    def manhattan_distance_from(self, other: Point) -> int:
        """ Manhattan distance between this Vector and another Vector """
        diff = self-other
        return Point.manhattan_distance(diff)

    def __repr__(self):
        return f"P({self.x},{self.y})"

class Vectors(Enum):
    """ Enumeration of 8 directions.
    Note: y axis increments in the North direction, i.e. N = (0, 1) """
    N = (0, 1)
    NE = (1, 1)
    E = (1, 0)
    SE = (1, -1)
    S = (0, -1)
    SW = (-1, -1)
    W = (-1, 0)
    NW = (-1, 1)

    @property
    def y_inverted(self):
        """ Return vector, but with y-axis inverted. I.e. N = (0, -1) """
        x, y = self.value
        return (x, -y)

class VectorDicts():
    """ Contains constants for Vectors """
    ARROWS = {
        '^': Vectors.N.value,
        '>': Vectors.E.value,
        'v': Vectors.S.value,
        '<': Vectors.W.value
    }

    DIRS = {
        'U': Vectors.N.value,
        'R': Vectors.E.value,
        'D': Vectors.S.value,
        'L': Vectors.W.value
    }

    NINE_BOX: dict[str, tuple[int, int]] = {
        # x, y vector for adjacent locations
        'tr': (1, 1),
        'mr': (1, 0),
        'br': (1, -1),
        'bm': (0, -1),
        'bl': (-1, -1),
        'ml': (-1, 0),
        'tl': (-1, 1),
        'tm': (0, 1)
    }

class Grid():
    """ 2D grid of point values. """
    def __init__(self, grid_array: list) -> None:
        self._array = grid_array
        self._width = len(self._array[0])
        self._height = len(self._array)

    def value_at_point(self, point: Point) -> int:
        """ The value at this point """
        return self._array[point.y][point.x]

    def set_value_at_point(self, point: Point, value: int):
        self._array[point.y][point.x] = value

    def valid_location(self, point: Point) -> bool:
        """ Check if a location is within the grid """
        if (0 <= point.x < self._width and  0 <= point.y < self._height):
            return True

        return False

    @property
    def width(self):
        """ Array width (cols) """
        return self._width

    @property
    def height(self):
        """ Array height (rows) """
        return self._height

    def all_points(self) -> list[Point]:
        points = [Point(x, y) for x in range(self.width) for y in range(self.height)]
        return points

    def rows_as_str(self):
        """ Return the grid """
        return ["".join(str(char) for char in row) for row in self._array]

    def cols_as_str(self):
        """ Render columns as str. Returns: list of str """
        cols_list = list(zip(*self._array))
        return ["".join(str(char) for char in col) for col in cols_list]

    def __repr__(self) -> str:
        return f"Grid(size={self.width}*{self.height})"

    def __str__(self) -> str:
        return "\n".join("".join(map(str, row)) for row in self._array)

### Useful Helper Functions

In [8]:
#################################################################
# CONSOLE STUFF
#################################################################

def cls():
    """ Clear console """
    os.system('cls' if os.name=='nt' else 'clear')

#################################################################
# USEFUL FUNCTIONS
#################################################################

def binary_search(target, low:int, high:int, func, *func_args, reverse_search=False):
    """ Generic binary search function that takes a target to find,
    low and high values to start with, and a function to run, plus its args.
    Implicitly returns None if the search is exceeded. """

    res = None  # just set it to something that isn't the target
    candidate = 0  # initialise; we'll set it to the mid point in a second

    while low < high:  # search exceeded
        candidate = int((low+high) // 2)  # pick mid-point of our low and high
        res = func(candidate, *func_args) # run our function, whatever it is
        logger.debug("%d -> %d", candidate, res)
        if res == target:
            return candidate  # solution found

        comp = operator.lt if not reverse_search else operator.gt
        if comp(res, target):
            low = candidate
        else:
            high = candidate

def merge_intervals(intervals: list[list]) -> list[list]:
    """ Takes intervals in the form [[a, b][c, d][d, e]...]
    Intervals can overlap.  Compresses to minimum number of non-overlapping intervals. """
    intervals.sort()
    stack = []
    stack.append(intervals[0])

    for interval in intervals[1:]:
        # Check for overlapping interval
        if stack[-1][0] <= interval[0] <= stack[-1][-1]:
            stack[-1][-1] = max(stack[-1][-1], interval[-1])
        else:
            stack.append(interval)

    return stack

@cache
def get_factors(num: int) -> set[int]:
    """ Gets the factors for a given number. Returns a set[int] of factors.
        # E.g. when num=8, factors will be 1, 2, 4, 8 """
    factors = set()

    # Iterate from 1 to sqrt of 8,
    # since a larger factor of num must be a multiple of a smaller factor already checked
    for i in range(1, int(num**0.5) + 1):  # e.g. with num=8, this is range(1, 3)
        if num % i == 0: # if it is a factor, then dividing num by it will yield no remainder
            factors.add(i)  # e.g. 1, 2
            factors.add(num//i)  # i.e. 8//1 = 8, 8//2 = 4

    return factors

def to_base_n(number: int, base: int):
    """ Convert any integer number into a base-n string representation of that number.
    E.g. to_base_n(38, 5) = 123

    Args:
        number (int): The number to convert
        base (int): The base to apply

    Returns:
        [str]: The string representation of the number
    """
    ret_str = ""
    curr_num = number
    while curr_num:
        ret_str = str(curr_num % base) + ret_str
        curr_num //= base

    return ret_str if number > 0 else "0"


# Env Clear

Only run the next cell if you want to manually clear your session key.

In [ ]:
del os.environ['AOC_SESSION_COOKIE']

# Days

Here you'll find a template to build a solution for a given day, and then the solutions for all days in this event.

To copy the template day, select all the cells in the `Day n` template, add a new cell at the end, and then paste the cells there.

---
## Day 1: Historian Hysteria

In [ ]:
DAY = "1" # replace with actual number (without leading digit)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2024d01
locations = dc.get_locations(d_name)
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", dc.top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day 1 Part 1

We're told: historically significant locations are listed by a unique number called the location ID. The Historians have split into two groups, and each group has created their own list of locations.

Our puzzle input is the two lists; i.e. presented as two columns of numbers. We need to reconcile the differences. E.g.

```text
3   4
4   3
2   5
1   3
3   9
3   3
```

Goal:

- Pair the smallest number in the two lists, and measure the difference.
- Then the second smallest number from each list, and so on.
- Return the sum of differences.

**What is the total distance between your lists?**

#### Solution Approach

1. Build two lists by splitting each line into two numbers. We can easily do this by [reading each line](https://aoc.just2good.co.uk/python/reading_files), and calling the `split()` function, which automatically splits on the whitespace between the numbers. For each row, we now end up with two numbers, but returned as string values. So we use [map()](https://aoc.just2good.co.uk/python/map-filter-reduce) to turn the strings into integer types. Now we have a pair of integer numbers for each line, which we wrap in a tuple and return in a [list comprehension](https://aoc.just2good.co.uk/python/comprehensions). I.e. one list containing tuples, which are each a pair of integers. Finally, we use the [zip()](https://aoc.just2good.co.uk/python/zip) function to turn the list of tuples pairs into a pair of lists: one list for left column, and one list for the right column.
1. Sort the two lists, such that the smallest number in each column comes first, and so on.
1. Find the absolute difference between each pair of numbers, and sum these differences.

In [ ]:
@cache
def process_data(data) -> tuple[list[int], list[int]]:
    items = [tuple(map(int, line.split())) for line in data] # returns list of tuples, e.g. [(3, 5), ...]
    list1, list2 = zip(*items) # list of all lefts, plus list of all rights
    return sorted(list1), sorted(list2)

def solve_part1(data) -> int:
    list1, list2 = process_data(tuple(data))
    
    diff = 0
    for item1, item2 in zip(list1, list2):
        diff += abs(item2 - item1)
        
    return diff

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""\
3   4
4   3
2   5
1   3
3   9
3   3
""")
sample_answers = [11]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("All tests passed!")

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day 1 Part 2

Figure out exactly how often each number from the left list appears in the right list. Calculate a total similarity score by adding up each number in the left list after multiplying it by the number of times that number appears in the right list.

**What is their similarity score?**

#### Solution Approach

1. For each item in list1, count how many times it appears in list2.
1. Add the product of the item and the count to our score.


In [ ]:
def solve_part2(data):
    list1, list2 = process_data(tuple(data))
    
    score = 0
    for item in list1:
        item_count = list2.count(item)
        score += item * item_count
        
    return score

In [ ]:
%%time

sample_answers = [31]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")    

logger.info("Tests passed!")

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")

### Alternative Solutions

We could also solve using [Numpy](https://aoc.just2good.co.uk/python/numpy). This will be more efficient for large arrays.

In [ ]:
def solve(data) -> int:
    sorted_lists = process_data(tuple(data))
    
    # Create two Numpy arrays
    list1 = np.array(sorted_lists[0])
    list2 = np.array(sorted_lists[1])
    
    diff = np.sum(np.abs(list2-list1)) # The sum of item diffs
    logger.info(f"{diff=}")
    
    similarity_score = 0
    for item in list1:
        item_count = (list2 == item).sum() # How many times the item appears in list2
        similarity_score += item * item_count
        
    logger.info(f"{similarity_score=}")
    
solve(input_data)

---
## Day 2: Red-Nosed Reports

In [ ]:
DAY = "2" # replace with actual number (without leading digit)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2024d01
locations = dc.get_locations(d_name)
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", dc.top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day 2 Part 1

Our data is a set of reports. Each line is one report. Each report contains a list of numbers called levels. Like this:

```text
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
```

Reports are safe if:

- The levels are either all increasing or all decreasing
- And any two adjacent levels must differ by at least 1 and at most 3.

**How many reports are safe?**

#### Solution Approach

- Split each record into a list of int values, called `levels`.
- Create `is_safe()` function which:
  - Creates a list of difference values - called `diffs` - by subtracting `levels` with offset 1 from `levels`.
  - Then, use the `all` iterator operator function to check if our condition is true for every diff in `diffs`.
  - The condition is: that every diff is positive and `<=MAX_DIFF` or every diff is negative and `>=MAX_DIFF`.

In [ ]:
def is_safe(levels: list[int], max_diff: int) -> bool:
    """ Determine if a record - i.e. a list of levels - is safe.
    Do this by determining the differences between each level in the record.
    Check if the differences are all positive and within the max, 
    or all negative and within the max. """
    
    # Create pairs of levels from successive levels
    # For each pair, substract second from first to get the difference
    diffs = [first-second for first, second in zip(levels, levels[1:])]
    
    return (all(0 < diff <= max_diff for diff in diffs) or 
            all(-max_diff <= diff < 0 for diff in diffs))

def solve_part1(data):
    safe_count = 0
    for report in data:
        levels = list(map(int, report.split()))
        if is_safe(levels, 3):
            safe_count += 1
    
    return safe_count

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""\
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9""")
sample_answers = [2]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("All tests passed!")

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day 2 Part 2

We now have a _Problem Dampener_, which means we can tolerate a single bad level in any given record.

So we must try removing levels from records, and then testing if the report is safe.

**How many reports are now safe?**

#### Solution Approach

Same as before, but this time we will iterate through the index values of each record, and remove that level from the record. I.e.

- Loop through each index value for the record.
- For the current index value, concatenate the levels up to this index with the levels after this index. Thus, we always end up with a list that is one shorter than the original list. (Note that trimming off the first or last level will never make a safe record _unsafe_.)
- Pass shortened list to our `is_safe()` function.
- If the shortened record is safe, then we can add this record to our counter and move on to the next record.

In [ ]:
def solve_part2(data):
    safe_count = 0
    for report in data:
        levels = list(map(int, report.split()))

        # Slice out one level at a time, and check if the new record is safe
        for idx in range(len(levels)):
            # Take levels up this idx, and concatenate with levels AFTER this idx
            trimmed_levels = levels[:idx] + levels[idx+1:]
            if is_safe(trimmed_levels, 3):
                safe_count += 1
                break
    
    return safe_count

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""\
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9""")
sample_answers = [4]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("Tests passed!")

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")

---
## Day 3: Mull It Over

In [ ]:
DAY = "3" # replace with actual number (without leading digit)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2024d01
locations = dc.get_locations(d_name)
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read()

    logger.info("Input data:\n%s", dc.top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day 3 Part 1

A shop computer is trying to run a program, but its memory - the puzzle input - is corrupted.

It should be doing instructions like:

- `mul(X,Y)` where `X` and `Y` are 1-3 digit numbers.

Our instructions:

- There are many invalid characters which should be ignored.
- If an invalid character is part of a `mul()` instruction, then the whole instruction does nothing.
- Scan the corrupted memory for uncorrupted mul instructions.

**What do you get if you add up all of the results of the multiplications?**

#### Solution Approach

We can just use [regex](https://aoc.just2good.co.uk/python/regex).

- Use a regex pattern to identify substrints that match the required input format.
- Use the regex `finditer()` to retrieve all non-overlapping matches.
- The `matches.groups()` retrieves the captured groups, i.e. the digits themselves. We identify capture groups by placing each digit in brackets in the pattern string.
- Then, multiply the two digits together, as required.

In [ ]:
def solve_part1(data):
    # match all "mul(x,y)"
    matches = re.finditer(r"mul\((\d{1,3}),(\d{1,3})\)", data)
    
    ans = 0
    for match in matches:
        val_x, val_y = match.groups()
        ans += int(val_x)*int(val_y)
        
    return ans


In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))""")
sample_answers = [161]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("All tests passed!")

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day 3 Part 2

Now we also want to handle intact conditional statements. We need to handle:

- `do()` which enables future `mul` instructions
- `don't()` which disables future `mul` instructions.

We start enabled.

**What is the sum of just the enabled multiplications?**

#### Solution Approach

- Okay, we start `enabled`.
- Whilst `enabled==True`, everything up to the next `don't()` should be parsed for `mul()` instructions.
- When we hit a `don't()`, we set `enabled=False`.
- Now, everything up to the next `do()` should be ignored.
- When we hit a `do()`, we set `enabled=True` again.

I'm shocked... This worked first time with no bugs!!

In [ ]:
def solve_part2(data: str):
    # match all "mul(x,y)"
    matcher = re.compile(r"mul\((\d{1,3}),(\d{1,3})\)")
    
    remaining_program = data
    enabled = True # We start enabled
    ans = 0
    while remaining_program:
        if enabled:
            # split into the part before the next "don't()", and everything afterwards
            this_part, _, remaining_program = remaining_program.partition(r"don't()")
        
            for match in matcher.finditer(this_part):
                val_x, val_y = match.groups()
                ans += int(val_x)*int(val_y)
            
            enabled = False
        else:
            # split into the part before the next "do()", and everything afterwards
            this_part, _, remaining_program = remaining_program.partition(r"do()")
            enabled = True
        
    return ans

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))""")
sample_answers = [48]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input), curr_ans) # test with sample data
    logger.info("Test passed")    

logger.info("Tests passed!")

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")

---
## Day 4: Ceres Search

In [9]:
DAY = "4" # replace with actual number (without leading digit)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

#### See [Day 4](https://adventofcode.com/2024/day/4).

In [10]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2024d01
locations = dc.get_locations(d_name)
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", dc.top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

17:15:33.391:aoc2024 - DBG: input.txt already exists
17:15:33.393:aoc2024 - INF: Input data:
  1: XXASMSMSXMXSMMXMAXMMMAMSAMASMSAMXMAXSXMASMAMMXXMASASXAMAMXSXMMSXSAMXSSXXMXAAXMAXMXXSXMXMMAAMMMMMMMMMMXXMSMSSSMXAAMAMMMSAMXXXXSMSAXMMXXMASMSS
  2: ASMMAAAMAMASAXSSMMSASAMXAAAAXXXMAXXAMSASXMASMSSMSAMSAMMSXMAXSMAAMMSAMAAMSMSMSSSSXSXSMSAAXMXMAAAAMMSAMXSXMAAAAAXMAMAMAMAMMXMSASAXMXSAMXSAMXAS
  3: XSAMSMMSAMASXMXAAXSASXXSSMXXXMMSMSMSMAMMASASXAAXXAMXAXAMAXMASMMSMAMMSMSMAAAAAAAAASASASXSMSASXSSXSAXAMAMAMMMSMMMMAXASMSMXSAAMAMXMMAMAXXAMXMMS
  4: AMAMXAASMMXSAMSMMMMMMMMMAMMSMMAAAAAAXAXSAMASMSSMSAMSAMMSMMMXXAAAMXMAAAAMXMMMMSMMMMAMAMAMAMMSXMAXMASXMSSMSAMXAAASMSMXAAXAMMSMAAMAMXSMMSASXSXM
  5: XSAMMMMXAXSMXXAMAAXSAAXSAMASAAMMSMSMSMMMSMAMMAAXMAMSASXAMXXXSMMMSAMMMSMSSSXXAXXAXMXMXMMMSMMMMMSMMAMAAAAXSMSSMMMMAAAMSMMXSAXXXMSXMAMAAXMAXMAX
...
136: MAMMXMAMMSMMMSMMASXMAXAAXXMASASMAMMMMSMMMMMXMXAXMSMSMSMSXXXAAXSXXSASAAXMMMAMMAMAAAMMMMSAMAMMMAAMMMSXAXAMAMXMXMMSMSXXAAMMXMASMAAAAMMMMMAMXMAS
137: SAMSAMXMMASAXASAXMAMXS

### Day 4 Part 1

We have a word search! We need to find every instance of the word `XMAS`.



In [11]:
def solve_part1(data):
    match_word = "XMAS"
    matched_counts = defaultdict(int)

    grid_width = len(data[0])
    grid_height = len(data)

    matches = 0

    for row_num in range(grid_height):
        for col_num in range(grid_width):
            # Find the X (first char)
            if data[row_num][col_num] == match_word[0]:
                # Now we want to expand in all directions, one direction at a time
                for direction in Vectors:
                    
                    # Check if the word will fit in the bounds
                    chars_left = len(match_word)-1
                    if not (0 <= (col_num + chars_left*direction.value[0]) < grid_width):
                        continue # try next direction
                    if not (0 <= (row_num + chars_left*direction.value[1]) < grid_height):
                        continue # try next direction

                    found_word = True

                    # Test remaining letters
                    for steps, char in enumerate(match_word[1:], start=1):
                        new_row = row_num + steps*direction.value[1]
                        new_col = col_num + steps*direction.value[0]
                        if data[new_row][new_col] != char:
                            found_word = False
                            break

                    if found_word:
                        # Mark these locations as matched
                        for steps, char in enumerate(match_word):
                            new_row = row_num + steps*direction.value[1]
                            new_col = col_num + steps*direction.value[0]
                            matched_counts[(new_col, new_row)] += 1
                        matches += 1
    
    grid_rows = []
    for row_num in range(grid_height):
        row_str = ""
        for col_num in range(grid_width):
            row_str += data[row_num][col_num] if matched_counts[(col_num, row_num)] > 0 else "."
        
        grid_rows.append(row_str)
    
    grid_vis = "\n".join(grid_rows)
    logger.info(f"\n{grid_vis}")

    return matches
    


In [12]:
%%time
sample_inputs = []
sample_inputs.append("""\
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX""")
sample_answers = [18]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("All tests passed!")

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

17:15:39.492:aoc2024 - INF: 
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
17:15:39.492:aoc2024 - INF: Test passed
17:15:39.493:aoc2024 - INF: All tests passed!
17:15:39.566:aoc2024 - INF: 
.......SX.X......X...........SAMX....XMAS.....XMAS..X.........S.SAMXSS...............................X..S.....X.......SAMXX..........XMAS.SS
......A.AM.....SM...SAMX.....X...XX.....XMAS.......S.M.S....S..A...A..A.S.S..SSS.S.S.S..X.........SAMX.X.A....XM......A...M..S...XSAMXSAMXA.
XS...M.SAMA.X..AA..........X..M..SMS................A.A.AXMAS..SM.M.S.SM.AAAAAA.A.ASAS..MS.....X...A....M.M..M..A.....M...A.A...MA.A.X...MM.
.MA.X.AS..XS.MS..M.........SM..A..AAX.X............S.M.S.MM...A..X..AAA.X.MMM.MM.MAMA...A...X..XMAS...S.SAMXA....S....X...SM...AMXS.M...X.X.
..AM.M....S..XA...X.......A..A..S.SMS..MS...........A.X..XX..M..S...M.M.S.XX.XX.XMXMXM.MS..MMM.M.A...A....SS........S.....X...SXM..A.X......
..ASX......A..MS........XMAS..S...AXX..AA

CPU times: total: 15.6 ms
Wall time: 77.3 ms


### Day 4 Part 2

Quite different for part 2! Now we need to find any any X shape that contains "MAS" twice.

#### Solution Approach

Let's move a 3x3 grid through our array, and look for the required chars in the X shape. E.g.

```text
M.S
.A.
M.S
```

- Fix relative grid positions for NE, SE, SW, NW.
- Move through every point in the grid, starting at (1, 1), rather than (0, 0).
- In our X grid, we need to check that:
  - The center letter must be `A`
  - From the four corners, the remaining two letters each appear twice. AND they must spell "MAS" along any diagonal.


In [43]:
def solve_part2(data):
    match_word = "MAS"
    center = match_word[1]
    ends = match_word[0] + match_word[-1]

    grid_width = len(data[0])
    grid_height = len(data)

    matches = 0

    corner_vecs = [Vectors.NE.value,
                   Vectors.SE.value,
                   Vectors.SW.value,
                   Vectors.NW.value]

    # Traverse the grid, but always inset by 1
    for row_num in range(1, grid_height-1):
        for col_num in range(1, grid_width-1):
            # Does middle of the X contain any of "MAS"?
            current_centre = data[row_num][col_num]
            if current_centre == center:
                corners = [(col_num+dx, row_num+dy) for dx, dy in corner_vecs]
                corner_chars = [data[y][x] for x, y in corners]
                if all(corner_chars.count(char) == 2 for char in ends):
                    logger.debug("Matched.")
                    matches += 1

    return matches

In [44]:
%%time
sample_inputs = []
sample_inputs.append("""\
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX""")
sample_answers = [9]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")    

logger.info("Tests passed!")

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")

17:50:09.092:aoc2024 - DBG: Matched.
17:50:45.205:aoc2024 - DBG: Matched.
17:50:45.207:aoc2024 - DBG: Matched.
17:50:45.208:aoc2024 - DBG: Matched.
17:50:45.209:aoc2024 - DBG: Matched.
17:50:45.210:aoc2024 - DBG: Matched.
17:50:45.211:aoc2024 - DBG: Matched.
17:50:45.212:aoc2024 - DBG: Matched.
17:50:45.213:aoc2024 - DBG: Matched.
17:50:45.215:aoc2024 - INF: Test passed
17:50:45.216:aoc2024 - INF: Tests passed!
17:50:45.217:aoc2024 - DBG: Matched.
17:50:45.218:aoc2024 - DBG: Matched.
17:50:45.218:aoc2024 - DBG: Matched.
17:50:45.219:aoc2024 - DBG: Matched.
17:50:45.221:aoc2024 - DBG: Matched.
17:50:45.222:aoc2024 - DBG: Matched.
17:50:45.223:aoc2024 - DBG: Matched.
17:50:45.224:aoc2024 - DBG: Matched.
17:50:45.225:aoc2024 - DBG: Matched.
17:50:45.225:aoc2024 - DBG: Matched.
17:50:45.226:aoc2024 - DBG: Matched.
17:50:45.226:aoc2024 - DBG: Matched.
17:50:45.227:aoc2024 - DBG: Matched.
17:50:45.227:aoc2024 - DBG: Matched.
17:50:45.228:aoc2024 - DBG: Matched.
17:50:45.229:aoc2024 - DBG: Ma

CPU times: total: 1.53 s
Wall time: 1min 3s


---
## Day n: title

In [ ]:
DAY = "n" # replace with actual number (without leading digit)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2024d01
locations = dc.get_locations(d_name)
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", dc.top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day n Part 1

Overview...

In [ ]:
def solve_part1(data):
    pass

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""abcdef""")
sample_answers = ["uvwxyz"]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("All tests passed!")

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day n Part 2

Overview...

In [ ]:
def solve_part2(data):
    pass

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""abcdef""")
sample_answers = ["uvwxyz"]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input), curr_ans) # test with sample data
    logger.info("Test passed")    

logger.info("Tests passed!")

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")